#Titanic: Machine Learning from Disaster
Olivier RISSER-MAROIX (VieVie31)

In [1]:
import graphlab as gl

In [2]:
data_train = gl.load_sframe("train.csv")
data_test = gl.load_sframe("test.csv")

2016-03-18 15:15:53,119 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.4 started. Logging: /tmp/graphlab_server_1458310552.log


Finished parsing file /Users/mac/Desktop/titanic/train.csv

Parsing completed. Parsed 100 lines in 0.029997 secs.

This non-commercial license of GraphLab Create is assigned to orissermaroix@gmail.com and will expire on March 12, 2017. For commercial licensing options, visit https://dato.com/buy/.
------------------------------------------------------

Finished parsing file /Users/mac/Desktop/titanic/train.csv

...

Parsing completed. Parsed 100 lines in 0.010295 secs.


Inferred types from first line of file as 
column_type_hints=[int,int,int,str,str,float,int,int,str,float,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
------------------------------------------------------

Finished parsing file /Users/mac/Desktop/titanic/test.csv

Parsing completed. Parsed 418 lines in 0.008131 secs.


Inferred types from first line of file as 
column_type_hints=[int,int,str,str,float,int,int,str,float,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


In [3]:
data_train.head(3)

Columns:
	PassengerId	int
	Survived	int
	Pclass	int
	Name	str
	Sex	str
	Age	float
	SibSp	int
	Parch	int
	Ticket	str
	Fare	float
	Cabin	str
	Embarked	str

Rows: 3

Data:
+-------------+----------+--------+-------------------------------+--------+
| PassengerId | Survived | Pclass |              Name             |  Sex   |
+-------------+----------+--------+-------------------------------+--------+
|      1      |    0     |   3    |    Braund, Mr. Owen Harris    |  male  |
|      2      |    1     |   1    | Cumings, Mrs. John Bradley... | female |
|      3      |    1     |   3    |     Heikkinen, Miss. Laina    | female |
+-------------+----------+--------+-------------------------------+--------+
+------+-------+-------+------------------+---------+-------+----------+
| Age  | SibSp | Parch |      Ticket      |   Fare  | Cabin | Embarked |
+------+-------+-------+------------------+---------+-------+----------+
| 22.0 |   1   |   0   |    A/5 21171     |   7.25  |       |    S     |
| 38.0 |   1   |   0   |     PC 17599     | 71.2833 |  C85  |    C     |
| 26.0 |   0   |   0   | STON/O2. 3101282 |  7.925  |       |    S     |
+------+-------+-------+------------------+---------+-------+----------+
[3 rows x 12 columns]

##Cleanning trainning data

In [4]:
data_train["male"] = data_train["Sex"] == "male"
data_train["female"] = data_train["Sex"] == "female"
data_train = data_train.remove_column("Sex")

In [5]:
data_train["no_age"] = data_train["Age"] == None
data_train["Age"] = gl.SArray([0 if v == None else v for v in data_train["Age"]])

In [6]:
data_train["embarked_s"] = data_train["Embarked"] == "S"
data_train["embarked_c"] = data_train["Embarked"] == "C"
data_train["embarked_q"] = data_train["Embarked"] == "Q"
data_train["embarked_none"] = data_train["Embarked"] == None
data_train = data_train.remove_column("Embarked")

In [12]:
data_train["1_class"] = data_train["Pclass"] == 1
data_train["2_class"] = data_train["Pclass"] == 2
data_train["3_class"] = data_train["Pclass"] == 3
data_train = data_train.remove_column("Pclass")

##Create model

In [7]:
help(gl.classifier.logistic_classifier.create)

Help on function create in module graphlab.toolkits.classifier.logistic_classifier:

create(dataset, target, features=None, l2_penalty=0.01, l1_penalty=0.0, solver='auto', feature_rescaling=True, convergence_threshold=0.01, step_size=1.0, lbfgs_memory_level=11, max_iterations=10, class_weights=None, validation_set='auto', verbose=True)
    Create a :class:`~graphlab.logistic_classifier.LogisticClassifier` (using
    logistic regression as a classifier) to predict the class of a discrete
    target variable (binary or multiclass) based on a model of class probability
    as a logistic function of a linear combination of the features.  In addition
    to standard numeric and categorical types, features can also be extracted
    automatically from list or dictionary-type SFrame columns.
    
    This model can be regularized with an l1 penalty, an l2 penalty, or both. By
    default this model has an l2 regularization weight of 0.01.
    
    Parameters
    ----------
    dataset : SFrame

In [17]:
train_set_1, train_set_2 = data_train.random_split(.8)

In [18]:
print train_set_1.head(1)
features = ["Age", "SibSp", "Parch", "Fare", "male", "female", "no_age", 
            "embarked_s", "embarked_c", "embarked_q", "embarked_none",
            "1_class", "2_class", "3_class"]

+-------------+----------+-------------------------------+------+-------+-------+
| PassengerId | Survived |              Name             | Age  | SibSp | Parch |
+-------------+----------+-------------------------------+------+-------+-------+
|      2      |    1     | Cumings, Mrs. John Bradley... | 38.0 |   1   |   0   |
+-------------+----------+-------------------------------+------+-------+-------+
+----------+---------+-------+------+--------+--------+------------+------------+
|  Ticket  |   Fare  | Cabin | male | female | no_age | embarked_s | embarked_c |
+----------+---------+-------+------+--------+--------+------------+------------+
| PC 17599 | 71.2833 |  C85  |  0   |   1    |   0    |     0      |     1      |
+----------+---------+-------+------+--------+--------+------------+------------+
+------------+---------------+---------+---------+---------+
| embarked_q | embarked_none | 1_class | 2_class | 3_class |
+------------+---------------+---------+---------+--------

In [19]:
simple_logistic_classifier = gl.classifier.logistic_classifier.create(train_set_1, target="Survived", 
                                                                      features=features, validation_set=train_set_2)

WARNING: Detected extremely low variance for feature(s) 'embarked_none' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.

Logistic regression:

--------------------------------------------------------

Number of examples          : 718

Number of classes           : 2

...

| 2         | 3        | 1.097328     | 0.798050          | 0.820809            |

| 3         | 4        | 1.100741     | 0.795265          | 0.820809            |

| 4         | 5        | 1.103611     | 0.795265          | 0.820809            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

##Cleanning testing data

In [20]:
data_test["male"] = data_test["Sex"] == "male"
data_test["female"] = data_test["Sex"] == "female"
data_test = data_test.remove_column("Sex")

In [21]:
data_test["no_age"] = data_test["Age"] == None
data_test["Age"] = gl.SArray([0 if v == None else v for v in data_test["Age"]])

In [22]:
data_test["embarked_s"] = data_test["Embarked"] == "S"
data_test["embarked_c"] = data_test["Embarked"] == "C"
data_test["embarked_q"] = data_test["Embarked"] == "Q"
data_test["embarked_none"] = data_test["Embarked"] == None
data_test = data_test.remove_column("Embarked")

In [23]:
data_test["1_class"] = data_test["Pclass"] == 1
data_test["2_class"] = data_test["Pclass"] == 2
data_test["3_class"] = data_test["Pclass"] == 3
data_test = data_test.remove_column("Pclass")

##Making Predictions

In [25]:
data_test["Survived"] = simple_logistic_classifier.predict(data_test)

In [27]:
submission = gl.SFrame()

In [28]:
submission["PassengerId"] = data_test["PassengerId"]
submission["Survived"] = data_test["Survived"]

In [29]:
submission.save("kaggle.csv", format="csv")